In [165]:
from keras.models import Sequential
from keras.layers import *

In [131]:
## Help Python find our packages
import sys
sys.path.append('..')

import json
import numpy as np
import matplotlib.pyplot as plt
import domain_scoring.domain_scoring as domain_scoring

# Randomness
import random as rn
import tensorflow as tf
import os

In [3]:
path = '../rated_datasets/Rotten_Tomato_Merlin_1519148528.2417703.json'

In [30]:
merlin = json.load(open(path, "r", encoding="utf8"))

In [5]:
path = '../rated_datasets/Rotten Tomato_Potato_1519142479.127663.json'

In [6]:
potato = json.load(open(path, "r", encoding="utf8"))

In [191]:
type_selection = merlin["edge_type_selection"] + merlin["node_type_selection"]
types = []
for pair in type_selection:
    types.append(pair[0])
types

['ACTED_IN', 'PRODUCED', 'DIRECTED', 'WROTE', 'Person', 'Movie']

In [86]:
def extract_mps(data):
    count = 0
    first = True
    batches = []
    batch = []
    for probably_path in data["meta_paths"]:
        if count % 6 == 0:
            # Don't add empty batches
            if len(batch) > 0:
                batches.append(batch)
                batch = []
        else:
            if 'time_to_rate' not in probably_path.keys():
                batch.append(probably_path)
        count += 1
    # append last batch
    if len(batch) > 0:
        batches.append(batch)
    print('#meta-paths:', count - len(batches) - 1)
    return batches

In [87]:
def construct_graph(batches):
    ## Construct rating graph
    from util.datastructures import MetaPathRatingGraph, MetaPath
    graph = MetaPathRatingGraph()

    for batch in batches:
        #ordered = sorted(batch, key=lambda x: float(x['rating']))
        for metapath in batch:
            for another_metapath in batch:
                if metapath is another_metapath:
                    continue
                if float(metapath['rating']) <= float(another_metapath['rating']):
                    graph.add_user_rating(MetaPath.from_list(another_metapath['metapath']), MetaPath.from_list(metapath['metapath']), 
                                          distance=float(another_metapath['rating']) - float(metapath['rating']))
    return graph

In [88]:
## Clean up data // remove time_to_rate from array of ratings.
def clean_up(data):
    batches = extract_mps(data)
    return batches, construct_graph(batches)

In [89]:
p_batches, p_graph = clean_up(potato)
m_batches, m_graph = clean_up(merlin)

#meta-paths: 51
#meta-paths: 51


In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [93]:
domain_score = domain_scoring.DomainScoring()

In [94]:
# Preprocess meta-paths
domain_score.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), token_pattern='\\b\\w+\\b')

In [95]:
domain_score.fit(m_graph)

In [175]:
x, y = domain_score._extract_data_labels(m_graph)
x_raw = x
x = domain_score._preprocess(x)

In [202]:
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

In [199]:
labeler = preprocessing.LabelEncoder()
labeler.fit(types)

LabelEncoder()

In [271]:
def to_sequence(in_list):
    return in_list.reshape((-1, 1))

In [272]:
def one_hot(in_sequence, distinct_values):
    sequence = np.zeros((len(in_sequence), distinct_values))
    i = 0
    for point in in_sequence:      
        sequence[i][point] = 1
        i += 1
    return sequence

In [299]:
def preprocess_raw(raw, distinct_values, max_len=9):
    data = []
    for a,b in raw:
        # encode labels to integers (0 is reserved for padding)
        a = np.array(labeler.transform(a.as_list())) + 1
        b = np.array(labeler.transform(b.as_list())) + 1
        # pad to same length
        a, b = pad_sequences([a, b], max_len, padding='post', value=0)
        # merge a and b
        sequence = np.append(a, b)
        # to sequence
        sequence = to_sequence(sequence)
        # one-hot encode because we don't have distances/embeddings
        sequence = one_hot(sequence, distinct_values + 1)
        data.append(sequence)
    return data

In [300]:
x_preprocess = preprocess_raw(x_raw, len(types))

In [97]:
test_size = 0.3
random_state = 42

In [98]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)

In [99]:
len(x_train)

131

In [105]:
np.array(x_train).shape

(131, 44)

In [173]:
os.environ['PYTHONHASHSEED'] = '0'
def reset_seed():
    np.random.seed(random_state)
    rn.seed(random_state)
    tf.set_random_seed(random_state)

In [319]:
# Reproducible results
reset_seed()
# Build model
model = Sequential()
model.add(Dense(44, activation='relu', input_dim=44))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(np.array(x_train), np.array(y_train), epochs=10, batch_size=16)

Epoch 1/10
131/131 [==============================] - 1s 4ms/step - loss: 0.6956 - acc: 0.5191
Epoch 2/10
131/131 [==============================] - 0s 106us/step - loss: 0.6822 - acc: 0.5878
Epoch 3/10
131/131 [==============================] - 0s 163us/step - loss: 0.6758 - acc: 0.5954
Epoch 4/10
131/131 [==============================] - 0s 141us/step - loss: 0.6717 - acc: 0.6183
Epoch 5/10
131/131 [==============================] - 0s 137us/step - loss: 0.6690 - acc: 0.6718
Epoch 6/10
131/131 [==============================] - 0s 149us/step - loss: 0.6659 - acc: 0.6794
Epoch 7/10
131/131 [==============================] - 0s 171us/step - loss: 0.6623 - acc: 0.6794
Epoch 8/10
131/131 [==============================] - 0s 145us/step - loss: 0.6610 - acc: 0.6565
Epoch 9/10
131/131 [==============================] - 0s 228us/step - loss: 0.6575 - acc: 0.6870
Epoch 10/10
131/131 [==============================] - 0s 204us/step - loss: 0.6559 - acc: 0.6794


In [320]:
model.evaluate(np.array(x_test), np.array(y_test), batch_size=16)

57/57 [==============================] - 0s 5ms/step


[0.6638993683614229, 0.6491228101546305]

In [321]:
x_p_train, x_p_test, y_p_train, y_p_test = train_test_split(x_preprocess, y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)

In [322]:
np.array(x_p_train).shape

(131, 18, 7)

In [323]:
# Reproducible results
reset_seed()
# Use sequence classification (RNN/LSTM)
model_rnn = Sequential()
model_rnn.add(SimpleRNN(128, input_shape=(18, 7)))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_rnn.fit(np.array(x_p_train), np.array(y_p_train), batch_size=35, epochs=10)

Epoch 1/10
131/131 [==============================] - 1s 5ms/step - loss: 0.7035 - acc: 0.5725
Epoch 2/10
131/131 [==============================] - 0s 390us/step - loss: 0.5776 - acc: 0.6641
Epoch 3/10
131/131 [==============================] - 0s 354us/step - loss: 0.5706 - acc: 0.6641
Epoch 4/10
131/131 [==============================] - 0s 340us/step - loss: 0.4936 - acc: 0.7252
Epoch 5/10
131/131 [==============================] - 0s 275us/step - loss: 0.5130 - acc: 0.7176
Epoch 6/10
131/131 [==============================] - 0s 256us/step - loss: 0.5067 - acc: 0.7328
Epoch 7/10
131/131 [==============================] - 0s 247us/step - loss: 0.5142 - acc: 0.7176
Epoch 8/10
131/131 [==============================] - 0s 272us/step - loss: 0.4981 - acc: 0.7481
Epoch 9/10
131/131 [==============================] - 0s 275us/step - loss: 0.4619 - acc: 0.7939
Epoch 10/10
131/131 [==============================] - 0s 257us/step - loss: 0.4563 - acc: 0.7634


In [324]:
model_rnn.evaluate(np.array(x_p_test), np.array(y_p_test), batch_size=35)

57/57 [==============================] - 0s 5ms/step


[0.5831738752231264, 0.7017544121073004]